# Reducing number of parameters of MobileNetV2 Model
This notebook shows how to use Low-Rank decomposition to reduce the number of parameters of a MobileNetV2 model. It assumes that a trained model already exist in the ```Models``` directory. Please refer to the notebook [Image Classification with MobileNetV2](MobileNetV2.ipynb) for more info about using a pretrained MobileNetV2 model.

## Load and evaluate the trained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus="0,1,2,3"   # Change this to match the GPUs available on your machine

testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)

model = Model.makeFromFile("Models/MobileNetV2.fbm", testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()

results = model.evaluate(topK=5)


Reading from "Models/MobileNetV2.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      864        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      288        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      512        
S2_L4_BN         112 112 16       

## Reducing number of parameters
Here we apply Low-Rank Decomposition on different layers of the model to reduce the number of parameters. We first create a list of layers we want to apply Low-Rank Decomposition, with the rank value for each layer. We then pass this information to the ```createLrModel``` method to create a new fireball model saved to the file ```Models/MobileNetV2R.fbm```.

In [2]:
# Here we want to reduce the number of parameters for layers "S8_L1_CONV" and "S8_L3_FC".
# Instead of specifying the tolerance (MSE), we are using spceific "rank" values for each
# one of these layers:
import time
layerParams = [('S8_L1_CONV', 184), ('S8_L3_FC', 304)]
print('Now reducing number of network parameters ... ')
t0 = time.time()
model.createLrModel("Models/MobileNetV2R.fbm", layerParams)
print('Done. (%.2f Seconds)'%(time.time()-t0))

Now reducing number of network parameters ... 
  S8_L1_CONV => LR(184), MSE=0.000260, Shape: (320, 1280), Params: 409600->294400 (Reduction: 28.1%)
  S8_L3_FC => LR(304), MSE=0.000467, Params: 1280000->693120 (Reduction: 45.9%)
Total New Parameters: 2,836,904
Done. (14.53 Seconds)


Compare the new number of parameters with the original 3,538,984. 

## Evaluating the new model
Let's see the impact of this reduction to the performance of the model.

In [3]:
model = Model.makeFromFile("Models/MobileNetV2R.fbm", testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()
results = model.evaluate(topK=5)


Reading from "Models/MobileNetV2R.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      864        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      288        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      512        
S2_L4_BN         112 112 16      

## Re-training
The following cell creates a "tune" dataset by sampling from the training dataset and uses it to re-train the  model for 10 epochs. For better results we could also use the whole training dataset for re-training.

If the trained model ```MobileNetV2RR.fbm``` is already available in the ```Models``` directory, this cell shows the results of last training. If you want to force it to do the training again, you can un-remark the lines at the beginning of the cell to delete the existing file. Note that the re-training can take up to 1 hour on a 4-GPU machine (Using the whole training dataset it could take up to 10 hours).

In [4]:
# Un-remark the following line if you want to delete the existing re-trained model and re-train it again.
# import os
# if os.path.exists( "Models/MobileNetV2RR.fbm" ): os.remove( "Models/MobileNetV2RR.fbm" )

tuneDs = ImageNetDSet.makeDatasets('tune', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)
print(tuneDs)

model = Model.makeFromFile("Models/MobileNetV2R.fbm", trainDs=tuneDs, testDs=testDs,
                           numEpochs=10,
                           learningRate=(0.0002,0.000002),
                           optimizer='Adam',
                           saveModelFileName="Models/MobileNetV2RR.fbm",  # Save the re-training state ...
                           savePeriod=1, saveBest=False,                  # ... every epoch
                           gpus=gpus)
model.printNetConfig()
model.initSession()
model.train()

ImageNetDSet Dataset Info:
    Dataset Name ................................... tune
    Dataset Location ............................... /data/ImageNet/
    Number of Classes .............................. 1000
    Number of Samples .............................. 64000
    Sample Shape ................................... (224, 224, 3)
    Preprocessing .................................. Crop256Tf
    Number of Workers .............................. 8


Reading from "Models/MobileNetV2R.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Network configuration:
  Input:                     Color images of size 224x224
  Output:                    Probability distributions for 1000 classes.
  Network Layers:            27
  Tower Devices:             GPU0, GPU1, GPU2, GPU3
  Total Network Parameters:  2,836,904
  Total Parameter Tensors:   264
  Trainable Tensors:         160
  Training Samples:          64,000
  Test Samples:              50,000
  Num Epochs:            

## Evaluate the model after re-training
Let's see how re-training helped with the performance of the model.

In [5]:
model = Model.makeFromFile("Models/MobileNetV2RR.fbm", testDs=testDs, gpus=gpus)
model.initSession()
results = model.evaluate(topK=5)


Reading from "Models/MobileNetV2RR.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.
  Processed 50000 Sample. (Time: 39.74 Sec.)                              

Observed Accuracy: 0.676120
Top-5 Accuracy:   0.882240


## Where do I go from here?

[Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)

[Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)

[Exporting MobileNetV2 Model to ONNX](MobileNetV2-ONNX.ipynb)

[Exporting MobileNetV2 Model to TensorFlow](MobileNetV2-TF.ipynb)
________________

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with MobileNetV2](MobileNetV2.ipynb)

